# import def func

In [1]:
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

from JohnsonUtil import LoggerFactory
log = LoggerFactory.log
# log_level = LoggerFactory.DEBUG
# log_level = LoggerFactory.INFO
log_level = LoggerFactory.ERROR
log.setLevel(log_level)
# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

log = LoggerFactory.log

In [2]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20,orderby='percent'):
    col_src = [
        "name",'percent', "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    if orderby in df.columns:
        df=df.sort_values('percent', ascending=False)
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

# from JSONData import tdx_hdf5_api as h5a
names = locals()
def get_tdx_all_now(resample='d',market_blk='all',showtable=True):
    top_now = tdd.getSinaAlldf(market=market_blk, vol=ct.json_countVol, vtype=ct.json_countType)
    if resample=='d' or resample=='3d':
        dl=ct.Resample_LABELS_Days[resample]
    elif  resample=='w':
        #200
        dl=ct.Resample_LABELS_Days[resample]
    else:
        #300
        dl=ct.Resample_LABELS_Days[resample]
    filter='y'
#     import pdb;pdb.set_trace()
    if not f'lastpTDX_DF_{resample}' in names:
        print(f'lastpTDX_DF_{resample} not in locals')
        top_all, lastpTDX_DF = tdd.get_append_lastp_to_df(top_now, dl=dl,resample=resample,showtable=showtable)
        names[f'lastpTDX_DF_{resample}'] = lastpTDX_DF
    else:
        print(f'lastpTDX_DF_{resample} in locals')
        print(eval(f'lastpTDX_DF_{resample}.shape'))
        top_all = tdd.get_append_lastp_to_df(top_now, eval(f'lastpTDX_DF_{resample}'))
    return top_all

def get_tdx_all_from_h5_old(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = tdd.get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


# def get_tdx_all_from_h5_old(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
#     if end is not None:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
#             '_' + end.replace('-', '') + '_' + 'all'
#     else:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     #sina_monitor
#     h5_fname = 'tdx_last_df'
#     resample=resample
#     if resample=='d' or resample=='3d':
#         dl='60'
#     elif  resample=='w':
#         dl='200'
#     else:
#         dl='300'
#     filter='y'
#     h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
#     return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [3]:
f'lastpTDX_DF_d' in names
f'lastpTDX_DF_d' in locals()
# eval(f'lastpTDX_DF_d')

False

False

## init data

### get_tdx_all

In [4]:
# h5_day=get_tdx_all_now(resample='d')

### sina

### sina_all

In [5]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-08-08',
 code
 002059    云南旅游
 Name: name, dtype: object)

In [6]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-08/10/25 15:59:29'

code
603018   2025-08-10 15:00:03
601956   2025-08-10 15:00:03
Name: ticktime, dtype: datetime64[ns]

59.433494973182675

#### 实时成交

In [7]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:G:\sina_MultiIndex_data.h5
[]
G:\sina_MultiIndex_data.h5


#### 读取实时成交个股

In [8]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

##### 合并实时成交量及SinaDM

In [9]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### filterKey!!!

In [10]:
filter_query={'2周期上冲上轨And涨幅新高': 'lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp',
             '3周期新高上轨':'lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3',
             '3周期新高大于Ma5': 'lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d',
             '新高上upper':'lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper',
            }   
filter_keys=list(filter_query.keys())

In [11]:
def get_filter(idx=0):
    filter_key=filter_query[filter_keys[idx]]
    print(f'{filter_keys[idx]}: {filter_key}')
    return filter_key

In [12]:
filterIDX=3
# h5_3d=None,h5_d=None,h5_w=None,h5_m=None

### resample 3d

In [13]:
# h5_3d = get_tdx_all_from_h5(resample='3d') if h5_3d is None else h5_3d
h5_3d = get_tdx_all_now(resample='3d')
# tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

initdx :0 b1>:5194 it:0.3 lastpTDX_DF_3d not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-08-10 15:59:31,263] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4572): tdx Out:4 code:['600837', '600225']


tdxdataT:2.75
DLTDXE:2.75 

In [14]:
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')
# t3d_code = h5_3d.query(f'{get_filter(filterIDX)}')
t3d_code = h5_3d.query('lasth1d > ma51d and lasth2d > ma52d and lasth3d > ma53d and ma51d > ma201d and ma52d >ma202d and ma53d > ma203d and macd > 0 and macddif > macddea')
show_tdx_data(t3d_code,limit=10)

Count:473 220


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
300780,德恩精工,20.03,50,2025-04-09,21.27,16.70,14.25,48.91,11.28,7.30,1.3
688551,科威尔,19.97,16,2025-04-09,43.19,31.62,26.91,56.91,22.69,1.10,1.3
300218,安利股份,13.27,28,2025-04-09,23.73,16.20,13.97,45.59,11.72,14.80,0.1
300289,利德曼,11.87,381,2025-04-09,10.93,8.45,4.76,31.30,3.85,39.31,1.3
688349,三一重能,11.09,6,2025-04-30,27.84,25.53,22.31,38.41,21.41,0.10,0.1
300455,航天智装,10.71,245,2025-04-25,19.75,14.53,12.26,50.91,11.73,1.10,1.3
601929,吉视传媒,10.20,236,2025-04-09,2.70,2.00,1.72,37.41,1.50,0.10,17.0
605169,洪通燃气,10.09,11,2025-04-09,16.91,12.26,9.89,55.19,8.13,1.30,0.0
603800,洪田股份,10.05,28,2025-05-23,42.47,31.70,20.97,48.31,20.97,1.20,21.2


In [15]:
if '301038' in t3d_code.index:
    h5_3d.loc['301038',['lastdu4','boll','ral','fib','percmax','macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

In [16]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# t3d_code = h5_3d.query(f'{get_filter(1)}')
# show_tdx_data(t3d_code)

### resample W

In [17]:
# h5_w = get_tdx_all_from_h5(resample='w') if h5_w is None else h5_w

h5_w = get_tdx_all_now(resample='w') 
# tdx_w = cct.combine_dataFrame(h5_w,sinadm)

initdx :0 b1>:5194 it:0.3 lastpTDX_DF_w not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-08-10 15:59:37,761] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4572): tdx Out:2 code:['600297', '600321']


tdxdataT:1.39
DLTDXE:1.39 

In [18]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tw_code = tdx_w.query(f'{get_filter(0)}')
# tw_code = tdx_w.query(f'{get_filter(filterIDX)}')
tw_code = h5_w.query(f'{get_filter(filterIDX)}')
# tw_code = h5_w.query('lastp2d > ma51d and lastp3d > ma51d and macd > 0 and macddif > macddea')
show_tdx_data(tw_code,limit=10)

新高上upper: lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper
Count:879 220


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
301112,信邦智能,20.05,2,2025-01-03,51.25,41.12,18.50,56.31,21.23,1.30,0.0
688577,浙海德曼,20.03,2,2024-11-22,69.81,47.25,26.50,71.62,23.78,34.91,-1.0
688108,赛诺医疗,20.02,13,2025-04-18,20.44,12.12,8.20,72.00,8.89,13.90,-0.9
688551,科威尔,19.97,16,2025-04-18,43.19,31.34,22.69,66.00,25.14,1.30,0.1
301232,飞沃科技,19.95,18,2025-01-03,42.09,32.06,20.02,56.81,22.77,0.10,0.0
688379,华光新材,16.84,10,2025-01-03,51.19,40.91,17.38,61.91,19.81,-0.90,0.0
301399,英特科技,13.21,10,2024-11-15,23.91,20.48,11.97,43.41,14.21,1.20,0.0
300732,设研院,12.78,114,2025-04-11,12.18,7.48,5.24,1.20,5.24,19.70,11.3
300289,利德曼,11.87,381,2025-04-11,10.93,5.56,3.85,80.31,3.85,11.10,1.3


In [19]:
if '301038' in tw_code.index:
    h5_w.loc['301038',['macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

### resample M

In [20]:
# h5_m = get_tdx_all_from_h5(resample='m') if h5_m is None else h5_m
h5_m = get_tdx_all_now(resample='m')
# tdx_m = cct.combine_dataFrame(h5_m,sinadm)

initdx :0 b1>:5194 it:0.3 lastpTDX_DF_m not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-08-10 15:59:42,911] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4572): tdx Out:2 code:['600321', '600297']


tdxdataT:1.61
DLTDXE:1.61 

In [21]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
#月线站上,3月反弹,2月新高
tm_code = h5_m.query('lastl1d > lastl2d > lastl3d and lasth1d > lastp2d > lastp3d and  ma51d > ma52d and macd > 0 and macddif > macddea')   
# tm_code = h5_m.query(f'{get_filter(filterIDX)}')
show_tdx_data(tm_code,limit=10)

Count:885 220


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
300780,德恩精工,20.03,50,2024-08-30,21.27,18.56,11.28,44.69,12.00,0.1,-1.1
688577,浙海德曼,20.03,2,2024-08-30,69.81,42.62,18.69,64.12,18.95,0.0,0.0
688108,赛诺医疗,20.02,13,2024-07-31,20.44,11.05,7.27,86.50,7.51,1.2,-1.1
301048,金鹰重工,20.00,77,2024-08-30,13.56,10.07,7.26,42.19,7.29,1.1,0.0
688265,南模生物,19.94,2,2024-08-30,51.91,27.55,19.42,73.69,19.59,24.7,-1.1
301038,深水规院,19.92,27,2024-08-30,34.97,16.12,9.53,1.10,9.53,1.3,-1.0
300218,安利股份,13.27,28,2025-04-30,23.73,17.38,11.72,66.00,11.72,1.2,-1.1
300732,设研院,12.78,114,2025-01-27,12.18,8.53,5.24,1.20,5.95,1.3,0.0
300289,利德曼,11.87,381,2024-06-28,10.93,5.52,3.56,100.62,3.70,1.3,0.0


In [22]:
if '301038' in tm_code.index:
    h5_m.loc['301038',['lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]
    
# h5_m.loc['301038',['lasth1d','lastp2d','lastp3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]    

lasth1d    34.97
lastl1d    27.95
lastl2d    15.37
lastl3d    13.92
lastp1d    34.97
ma51d      22.03
ma52d       18.0
ma53d      14.59
ma54d      13.97
Name: 301038, dtype: object

### resample D

In [23]:
# h5_d = get_tdx_all_from_h5(resample='d') if h5_d is None else h5_d
h5_d = get_tdx_all_now(resample='d')
# tdx_d = cct.combine_dataFrame(h5_d,sinadm)
h5_d.shape

initdx :0 b1>:5194 it:0.3 lastpTDX_DF_d not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-08-10 15:59:48,788] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4572): tdx Out:8 code:['600837', '600705']


tdxdataT:5.45
DLTDXE:5.45 

(5186, 220)

In [24]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
# td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')

# td_code = h5_d.query(f'{get_filter(filterIDX)}')
# td_code = h5_d.query(f'lasth1d > lasth2d and close > lastp1d and ral > 10 and fib > 0 and boll > 0')
td_code = h5_d.query(f'lasth1d > lasth2d  and ral > 10 and fib > 0 and boll > 0  and percent > lastdu4 \
and close >high4  and (low > open*0.99 or (per1d > 4 and low >= lastp1d))')

show_tdx_data(td_code,limit=20)
# if len(td_code) < 30:
#     write_code_to_blk(td_code.sort_values('percent', ascending=False).index,'065',rewrite=True)

Count:255 220


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
688577,浙海德曼,20.03,2,2025-06-23,69.81,54.81,43.22,45.91,41.59,11.20,4.1
688108,赛诺医疗,20.02,13,2025-05-19,20.44,14.65,10.65,42.31,9.71,-1.10,0.1
301048,金鹰重工,20.00,77,2025-05-23,13.56,11.38,10.34,42.31,9.82,1.20,1.3
688551,科威尔,19.97,16,2025-05-16,43.19,31.75,28.72,61.19,26.94,1.30,1.3
301232,飞沃科技,19.95,18,2025-05-23,42.09,33.59,29.50,55.00,27.05,1.30,1.3
301038,深水规院,19.92,27,2025-05-16,34.97,34.00,15.37,20.30,13.51,-1.00,-0.9
301399,英特科技,13.21,10,2025-05-30,23.91,20.38,18.31,39.19,16.34,24.91,1.3
688349,三一重能,11.09,6,2025-05-23,27.84,25.53,23.78,33.41,22.45,0.10,0.1
300455,航天智装,10.71,245,2025-05-23,19.75,17.03,13.47,45.09,12.29,0.10,16.0


In [25]:
if '301038' in td_code.index:
    h5_d.loc['301038',['lastdu4','close','boll','ral','fib','percmax','macd','macddif','macddea','macdlast1','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

lastdu4       1.15
close        34.97
boll             1
ral             16
fib              5
percmax       19.0
macd          3.95
macddif       0.29
macddea       3.66
macdlast1     3.95
macdlast2     3.71
macdlast3     3.94
lasth1d      34.97
lastl1d      28.81
lastl2d       28.2
lastl3d      27.95
lastp1d      34.97
ma51d        31.16
ma52d        30.48
ma53d        30.77
ma54d        31.75
Name: 301038, dtype: object

#### filter test

In [32]:
# h5_t_code=h5_w.query('lasth1d > upper1 and lasth1d >lasth2d   and high >boll and percent > maxp')
# h5_t_code=h5_d.query('lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d and lastp2d < upper2 and lasth1d >upper')
code_3f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index))

show_tdx_data(t3d_code.loc[code_3f],limit=10)

Count:24 220


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
300289,利德曼,11.87,381,2025-04-09,10.93,8.45,4.76,31.30,3.85,39.31,1.3
605169,洪通燃气,10.09,11,2025-04-09,16.91,12.26,9.89,55.19,8.13,1.30,0.0
603800,洪田股份,10.05,28,2025-05-23,42.47,31.70,20.97,48.31,20.97,1.20,21.2
002046,国机精工,9.99,289,2025-04-09,31.05,19.28,15.06,55.41,11.65,15.30,16.2
603131,上海沪工,9.97,127,2025-04-09,21.62,18.16,16.38,42.91,14.38,0.00,1.3
301397,溯联股份,7.56,2,2025-04-09,41.00,31.23,25.08,45.59,17.92,0.10,15.4
603516,淳中科技,3.67,19,2025-04-25,90.00,66.50,32.44,45.69,32.84,1.20,1.1
300342,天银机电,3.00,119,2025-04-09,19.20,17.89,15.32,1.20,12.94,1.30,11.4
002068,黑猫股份,2.62,29,2025-06-13,12.13,12.16,9.37,1.10,9.45,28.41,0.0


In [27]:
# td_code2 = tdx_d.query(f'{get_filter(2)}')
# td_code2 = h5_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
# show_tdx_data(td_code2,limit=10)


#### 多周期筛选

In [28]:
for co in ['tm_code','tw_code','t3d_code','td_code']:
    print(f'{co}:{len(eval(co))}')
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))
# code_f=  list(set(tm_code.index) & set(tw_code.index) & set(td_code.index))
code_f =td_code.loc[code_f].sort_values('percent', ascending=False).index.tolist()
if len(code_f) > 0:
    show_tdx_data(td_code.loc[code_f],limit=10)
else:
    print('no code')

tm_code:885
tw_code:879
t3d_code:473
td_code:255
Count:3 220


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
605169,洪通燃气,10.09,11,2025-05-23,16.91,12.69,10.55,39.50,10.28,17.3,1.3
603131,上海沪工,9.97,127,2025-05-23,21.62,18.73,17.17,39.09,16.41,1.3,0.1
300342,天银机电,3.00,119,2025-05-23,19.20,18.44,16.39,1.20,15.37,0.0,1.3


#### check data

In [29]:
'300998' in code_f
code_check = [co for co in code_f if co in h5_3d.index]
len(code_check)

False

3

In [30]:
show_tdx_data(td_code.loc[code_f])

Count:3 220


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
605169,洪通燃气,10.09,11,2025-05-23,16.91,12.69,10.55,39.50,10.28,17.3,1.3
603131,上海沪工,9.97,127,2025-05-23,21.62,18.73,17.17,39.09,16.41,1.3,0.1
300342,天银机电,3.00,119,2025-05-23,19.20,18.44,16.39,1.20,15.37,0.0,1.3


### Write toblk

In [31]:
# write_code_to_blk(code_f,'077',rewrite=False)